In [1]:
import sys
sys.path.append("/Users/rq.aita/Scripts")
sys.path.append("/home/ufpr/Documentos/GitHub/scripts")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import hydrology as hydro
import settings_plot

# Exercício 1

Um teste com um infiltrômetro de 35 cm de diâmetro produziu os seguintes resultados: 

| t (minutos) | Volume total adicionado (cm³) |
|-------------|-------------------------------|
| 0           | 0                             |
| 2           | 278                           |
| 5           | 658                           |
| 10          | 1.173                         |
| 20          | 1.924                         |
| 30          | 2.500                         |
| 60          | 3.345                         |
| 90          | 3.875                         |
| 150         | 4.595                         |

Pede-se traçar o gráfico f x t, onde f é a capacidade de infiltração dada em mm/h.

In [7]:
e_01_dados = {
    "t" : [0, 2, 5, 10, 20, 30, 60, 90, 150],
    "v" : [0, 278, 658, 1173, 1924, 2500, 3345, 3875, 4595],
}
e_01_dados = pd.DataFrame(e_01_dados)